In [25]:
# Extract financial data and historical stock prices from Yahoo! Finance using background java strings and a hidden api.

In [26]:
import re
import json
import csv
from io import StringIO
from bs4 import BeautifulSoup
import requests

In [42]:
# url templates
# Replace the stock symbol in the url with a curly brace to turn it into a template.
url_stats = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'

# the stock I want to scrape
stock = 'fb'

In [28]:
# request the webpage, passing in the stock variable to fill in the url template
response = requests.get(url_financials.format(stock, stock))

In [29]:
# extract the html using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser') # Beautiful Soup is a Python library for getting data out of HTML

In [30]:
# this the data we are trying to scrape:
# in google:  view-source:https://finance.yahoo.com/quote/FB/

# i can explore the dictionary to discover what's inside. This dataset contains both Annual and Quarterly financial statements, as you can see from the dictionary paths listed below.

<div>
<img src="webscrapedictionary.PNG" style="width: 800px;"/>
</div>

In [31]:
# there is a javascript function, appropriately commented with "--Data--".
# i can use regular expressions with BeautifulSoup in order to identify the script tag with the function i am looking for.
pattern = re.compile(r'\s--\sData\s--\s') # compile allows us to save regular expressiom
script_data = soup.find('script', text=pattern).contents[0]
# There's a lot of good json data here, but it's wrapped in a javascript function

In [32]:
#  identify the starting and ending position of this json data, we can slice it and then parse it with the json.loads function.

# beginning
script_data[:500]

# where context is - ... App.main = {"context":{"dispatcher" ... so we need 2 places before the c starts!

'\n(function (root) {\n/* -- Data -- */\nroot.App || (root.App = {});\nroot.App.now = 1621953320588;\nroot.App.main = {"context":{"dispatcher":{"stores":{"PageStore":{"currentPageName":"quote","currentEvent":{"eventName":"NEW_PAGE_SUCCESS"},"currentRenderTargetId":"default","pagesConfigRaw":{"base":{"quote":{"layout":{"bundleName":"yahoodotcom-layout.TwoColumnLayout","name":"TwoColumnLayout","config":{"enableHeaderCollapse":true,"additionalBodyWrapperClasses":"Bgc($layoutBgColor)!","contentWrapperClas'

In [33]:
# the end
script_data[-500:]

'put":{"strings":1},"tdv2-applet-sponsored-moments":{"strings":1},"tdv2-applet-stream":{"strings":1},"tdv2-applet-stream-hero":{"strings":1},"tdv2-applet-swisschamp":{"strings":1},"tdv2-applet-uh":{"strings":1},"tdv2-applet-userintent":{"strings":1},"tdv2-applet-video-lightbox":{"strings":1},"tdv2-applet-video-modal":{"strings":1},"tdv2-wafer-adfeedback":{"strings":1},"tdv2-wafer-header":{"strings":1},"yahoodotcom-layout":{"strings":1}}},"options":{"defaultBundle":"td-app-finance"}}}};\n}(this));\n'

In [34]:
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

In [35]:
# search until found data needed 
json_data['context']['dispatcher']
json_data['context']['dispatcher'].keys()

# ...

# data neede found in
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()



dict_keys(['financialsTemplate', 'cashflowStatementHistory', 'balanceSheetHistoryQuarterly', 'earnings', 'price', 'incomeStatementHistoryQuarterly', 'incomeStatementHistory', 'balanceSheetHistory', 'cashflowStatementHistoryQuarterly', 'quoteType', 'summaryDetail', 'symbol', 'pageViews'])

In [48]:
url_stats      = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile    = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'


# get data from yahoo / finance - explained deeply in yahoo-finance-scraper

response = requests.get(url_financials.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

#get stock price
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']

500,000,000'},
    'totalCashFromFinancingActivities': {'raw': -15572000000,
     'fmt': '-15.57B',
     'longFmt': '-15,572,000,000'},
    'changeToOperatingActivities': {'raw': 91000000,
     'fmt': '91M',
     'longFmt': '91,000,000'},
    'netIncome': {'raw': 22112000000,
     'fmt': '22.11B',
     'longFmt': '22,112,000,000'},
    'changeInCash': {'raw': 1920000000,
     'fmt': '1.92B',
     'longFmt': '1,920,000,000'},
    'endDate': {'raw': 1546214400, 'fmt': '2018-12-31'},
    'repurchaseOfStock': {'raw': -16087000000,
     'fmt': '-16.09B',
     'longFmt': '-16,087,000,000'},
    'effectOfExchangeRate': {'raw': -179000000,
     'fmt': '-179M',
     'longFmt': '-179,000,000'},
    'totalCashFromOperatingActivities': {'raw': 29274000000,
     'fmt': '29.27B',
     'longFmt': '29,274,000,000'},
    'depreciation': {'raw': 4315000000,
     'fmt': '4.32B',
     'longFmt': '4,315,000,000'},
    'otherCashflowsFromInvestingActivities': {'raw': -36000000,
     'fmt': '-36M',
     'lon

In [54]:
# find the starting position of the json string
start = script_data.find("context")-2 # location of context

# n}(this));\n' - is 12 in length
# slice the json string
json_data = json.loads(script_data[start:-12])

In [24]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']


NameError: name 'json_data' is not defined

In [22]:
marketCap = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['summaryDetail']['marketCap']['fmt']

json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

dict_keys(['defaultKeyStatistics', 'financialsTemplate', 'price', 'financialData', 'quoteType', 'calendarEvents', 'summaryDetail', 'symbol', 'pageViews'])

In [65]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']


{'cashflowStatements': [{'investments': {'raw': -14520000000,
    'fmt': '-14.52B',
    'longFmt': '-14,520,000,000'},
   'changeToLiabilities': {'raw': 91000000,
    'fmt': '91M',
    'longFmt': '91,000,000'},
   'totalCashflowsFromInvestingActivities': {'raw': -30059000000,
    'fmt': '-30.06B',
    'longFmt': '-30,059,000,000'},
   'netBorrowings': {'raw': -580000000,
    'fmt': '-580M',
    'longFmt': '-580,000,000'},
   'totalCashFromFinancingActivities': {'raw': -10292000000,
    'fmt': '-10.29B',
    'longFmt': '-10,292,000,000'},
   'changeToOperatingActivities': {'raw': -1302000000,
    'fmt': '-1.3B',
    'longFmt': '-1,302,000,000'},
   'netIncome': {'raw': 29146000000,
    'fmt': '29.15B',
    'longFmt': '29,146,000,000'},
   'changeInCash': {'raw': -1325000000,
    'fmt': '-1.32B',
    'longFmt': '-1,325,000,000'},
   'endDate': {'raw': 1609372800, 'fmt': '2020-12-31'},
   'repurchaseOfStock': {'raw': -9836000000,
    'fmt': '-9.84B',
    'longFmt': '-9,836,000,000'},
   '

In [17]:
# same process for statistics page

response = requests.get(url_stats.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

In [18]:
# data example from statistics file
peg = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['defaultKeyStatistics']['pegRatio']['raw']
peg


0.98